In [1]:
import pandas as pd
import numpy as np

In [2]:
headers = ['numbers', 'time', 'tweets']

In [3]:
import os
directory = r'C:\Users\SEGUN\Documents\New folder\Health-Tweets'
txtFiles = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
      txtFiles.append(filename)
      continue
    else:
        continue

df = pd.DataFrame() 

for txtFile in txtFiles:
    data = pd.read_csv(txtFile, delimiter = '|', names = headers, encoding='latin1')
    df = pd.concat((df,data))
    
df

,numbers,time,tweets
0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised http://bbc.in/...
1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll http://bbc...
2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater' http://bbc...
3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising' http://bb...
4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,Coalition 'undermined NHS' - doctors http://bb...
...,...,...,...
3195,415494259022655489,Tue Dec 24 14:48:45 +0000 2013,RT @stefaniei: Addiction and the brain: scient...
3196,415493351396233216,Tue Dec 24 14:45:09 +0000 2013,RT @timothywmartin: Ho-ho-hold up! A surprise ...
3197,415493203983204352,Tue Dec 24 14:44:33 +0000 2013,RT @stefaniei: Health-Insurance Deadline Exten...
3198,415386956420231169,Tue Dec 24 07:42:22 +0000 2013,Boston Scientific Eyes China Expansion http://...


In [5]:
df.to_csv('healthcare_tweet.csv')

In [6]:
df.shape

(55230, 3)

In [7]:
df.isnull().values.any()

False

In [8]:
import re
import string

from Scripts.preprocessor import TwitterPreprocessor

import nltk
from nltk import pos_tag, map_tag
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from textblob import TextBlob
from afinn import Afinn

In [10]:
df['clean_text'] = df.tweets.apply(lambda x: TwitterPreprocessor(x).ml_preprocess().text)

In [11]:
df.head()

,numbers,time,tweets,clean_text
0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised
1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll
2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater' http://bbc...,short people heart risk greater
3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising' http://bb...,new approach hiv promising
4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors


In [12]:
df.drop(['numbers', 'time'], axis=1, inplace=True)

In [13]:
df.head()

,tweets,clean_text
0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised
1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll
2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater
3,New approach against HIV 'promising' http://bb...,new approach hiv promising
4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors
